In [2]:
import pandas as pd 
df = pd.read_csv('preferences.csv')


In [3]:
dfList = df.values.tolist()
preferenceList = []
#for col in df.columns:
#    dfList.append(col)

for person in dfList:
    scores = person[0].split()
    listOfScores = []
    for s in scores:
        listOfScores.append(float(s))
    labour = min(listOfScores[0], listOfScores[4], listOfScores[7])
    brexit = listOfScores[1]
    libdems = min(listOfScores[2], listOfScores[5], listOfScores[6])
    green = listOfScores[3]
    cons = min(listOfScores[8], listOfScores[9], listOfScores[10])
    newList = [labour, brexit, libdems, green, cons]
    preferenceList.append(newList)

numParties = 5
numPlayers = len(preferenceList)
    

In [7]:
import random
import math
import matplotlib.pyplot as plt

def GenerateRandomOpinions(numParties, numPlayers):
    listOfPlayersOpinions = []  # list of lists containing initial preference info
    for player in range(numPlayers):
        playerOpinions = [0] * numParties
        for party in range(numParties):
            playerOpinions[party] = round(random.uniform(-10, 10),2)  # opinions range from -10 to +10, to 2dp
        if all(i < 0 for i in playerOpinions): #if person only has negative opinions
            randomSwitch = random.randint(0, numParties-1)
            playerOpinions[randomSwitch] = -playerOpinions[randomSwitch]
        listOfPlayersOpinions.append(playerOpinions)
    return listOfPlayersOpinions


def PartyPopularities(
        listOfOpinions, numParties):  # input list of list of opinions. output list with the number of people who prefer each party
    partyPopularity = [0] * numParties
    for player in range(len(listOfOpinions)):
        opinions = listOfOpinions[player]
        maxpref = -11
        index = 0
        for party in range(numParties):
            if opinions[party] > maxpref:
                maxpref = opinions[party]
                index = party
        partyPopularity[index] += 1
    return partyPopularity

def GameTheoryODE(playersOpinions, partyPopularities): # returns a new list of party opinions
    listOfOpinions = playersOpinions
    #print(listOfOpinions)
    maxvotes = max(partyPopularities)
    alpha = 0.4
    beta = 0.1
    for party in range(len(partyPopularities)):
        if partyPopularities[party] == maxvotes:
            winningParty = party

    for player in range(len(playersOpinions)):
        opinions = playersOpinions[player]
        newOpinions = playersOpinions[player]
        if opinions[winningParty] == max(opinions): # If your prefered party is likely to win, you should always vote for them
            newOpinions = opinions
        elif opinions[winningParty] < 0: # use weighting of how much you hate that party
            for party in range(len(partyPopularities)):
                newOpinions[party] += (opinions[party] - opinions[winningParty]) * (partyPopularities[party]/len(playersOpinions)) * alpha
        else:
            for party in range(len(partyPopularities)):
                newOpinions[party] += opinions[party] * (partyPopularities[party]/len(playersOpinions)) * beta
        listOfOpinions[player] = newOpinions
    #print(listOfOpinions)
    return listOfOpinions


def GenerateRandomPoints(numPoints, numDimensions):
    listOfPoints = []
    for point in range(numPoints):
        pointCentre = [0]*numDimensions
        for dimension in range(numDimensions):
            pointCentre[dimension] = round(random.uniform(-1, 1),2)
        listOfPoints.append(pointCentre)
    
    return listOfPoints


def CalculateDistanceToEachParty(point, listOfParties):
    listOfDistances = []
    for party in listOfParties:
        distance = 0
        for dim in range(len(point)):
            distance += abs(point[dim] - party[dim])**2
        listOfDistances.append(round(math.sqrt(distance),2))

    return listOfDistances


def PreferenceScoreFromDistance(distances):
    listOfPreferences = []
    minDistance = min(distances)
    n = 20/(max(distances)-minDistance)
    for dis in distances:
        prefScore = 10 - n*(dis-minDistance)
        listOfPreferences.append(round(prefScore,2))
    print(listOfPreferences)
    return listOfPreferences

In [8]:
def run(numParties, numPlayers):
    listOfPartyPop = []
    opinions = []
    for person in preferenceList:
        opinions.append(PreferenceScoreFromDistance(person))

    partyPop = PartyPopularities(opinions, numParties)
    listOfPartyPop.append(partyPop)
    for i in range(26):
        opinions = GameTheoryODE(opinions, partyPop)
        partyPop = PartyPopularities(opinions, numParties)
        listOfPartyPop.append(partyPop)
    listOfParties = [1,2,3,4,5,6,7,8,9,10,11]
    print(listOfPartyPop)
    

In [9]:
run(numParties, numPlayers)

[9.52, -10.0, -1.56, -3.15, 10.0]
[10.0, -10.0, -0.76, -3.0, 8.94]
[4.17, -10.0, 10.0, -4.4, 9.81]
[8.58, -10.0, -2.17, -3.32, 10.0]
[8.12, 10.0, 5.78, 0.1, -10.0]
[9.38, -10.0, 0.42, -2.5, 10.0]
[10.0, -10.0, -0.84, -3.05, 8.92]
[8.07, -10.0, 0.46, -3.19, 10.0]
[9.57, -10.0, -1.45, -3.17, 10.0]
[-0.49, -2.09, 2.07, 10.0, -10.0]
[4.1, -10.0, 10.0, -4.43, 9.77]
[8.58, -10.0, -1.92, -3.22, 10.0]
[10.0, -10.0, -0.87, -2.47, 8.77]
[10.0, 9.66, 6.42, 2.42, -10.0]
[10.0, 9.01, 5.85, 0.96, -10.0]
[5.02, 4.07, 10.0, 1.22, -10.0]
[10.0, -10.0, -1.17, -3.08, 9.13]
[8.4, 10.0, 5.71, -0.62, -10.0]
[8.46, 10.0, 5.44, -0.77, -10.0]
[8.16, 10.0, 5.71, -0.36, -10.0]
[9.54, 10.0, 6.48, 1.88, -10.0]
[8.02, 10.0, 5.12, -0.4, -10.0]
[9.13, 10.0, 6.06, 1.45, -10.0]
[10.0, 8.41, 7.42, 1.03, -10.0]
[8.74, 10.0, 4.98, -0.24, -10.0]
[0.42, -10.0, 6.79, -5.9, 10.0]
[10.0, 9.91, 6.27, 1.22, -10.0]
[10.0, -10.0, -0.82, -3.06, 9.11]
[9.26, -10.0, 0.59, -3.22, 10.0]
[9.27, -10.0, 0.62, -3.22, 10.0]
[1.6, -1.76, 10.